In [ ]:
import boto3
import os
from botocore.exceptions import ClientError

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = 'genai-accelerate-2024'
subdirectory = 'llama-3-2-vision-folder'
train_data_location = f"s3://{bucket_name}/{subdirectory}"

# Files to upload (metadata.jsonl and images folder)
files_to_upload = [
    '/home/ec2-user/SageMaker/vision-workshop/dataset/metadata.jsonl',
    '/home/ec2-user/SageMaker/vision-workshop/dataset/images'
]

for file_path in files_to_upload:
    if os.path.isfile(file_path):
        # Upload single file
        file_name = os.path.basename(file_path)
        key_path = f"{subdirectory}/{file_name}"
        try:
            s3.upload_file(file_path, bucket_name, key_path)
            print(f"File {file_name} uploaded successfully to {key_path}.")
        except ClientError as e:
            print(f"Error uploading file {file_name}: {e}")
    elif os.path.isdir(file_path):
        # Upload directory
        for root, _, files in os.walk(file_path):
            for file in files:
                full_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_file_path, start=file_path)
                s3_key = f"{subdirectory}/images/{relative_path}"
                try:
                    s3.upload_file(full_file_path, bucket_name, s3_key)
                    print(f"File {file} uploaded successfully to {s3_key}.")
                except ClientError as e:
                    print(f"Error uploading file {file} to {s3_key}: {e}")

In [ ]:
from sagemaker import hyperparameters

# Set model ID and version
model_id, model_version = "meta-vlm-llama-3-2-11b-vision-instruct", "*"

my_hyperparameters = hyperparameters.retrieve_default(
    model_id=model_id, model_version=model_version
)
print(my_hyperparameters)

In [ ]:
my_hyperparameters["epoch"] = "4"

In [ ]:
hyperparameters.validate(
    model_id=model_id, model_version=model_version, hyperparameters=my_hyperparameters
)